### Imports

In [21]:
from z3 import *
from itertools import combinations

### Constraint encodings

In [22]:
def gen_var_id():
    i = 0
    while True:
        i += 1
        yield i
gen_id = gen_var_id()

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def at_least_one_he(bool_vars):
    return Or(bool_vars)

def at_most_one_he(bool_vars, n = 4):
    formula, sub_seq = [], [0]*n
    i, j, _len = 0, 0, len(bool_vars)
    id = next(gen_id)
    for k in range(_len):
        sub_seq[i] = bool_vars[k]
        if _len - k == 2:
            sub_seq[i+1] = bool_vars[-1]
            formula.append(at_most_one_np(sub_seq[:i+2]))
            break
        elif i == n-2:
            sub_seq[-1] = Bool(f"y_{j}__{id}")
            formula.append(at_most_one_np(sub_seq))
            sub_seq[0] = Not(sub_seq[-1])
            j += 1
            i = 0
        i += 1
    return And(formula)

def at_most_k_seq(x, k):
    id = next(gen_id)
    n = len(x)
    if n <= k:
        return True
    s = [[Bool(f"s_{i}_{j}__{id}") for j in range(k)] for i in range(n-1)]
    return (
        And(
            Implies(x[0], s[0][0]),
            Implies(s[n-2][k-1], Not(x[n-1])),
            And([Not(s[0][j]) for j in range(1, k)]),
            And([ 
                And(
                    Implies(Or(x[i], s[i-1][0]), s[i][0]),
                    And([
                        Implies(Or(And(x[i], s[i-1][j-1]), s[i-1][j]), s[i][j])
                    for j in range(1, k)
                    ]),
                    Implies(s[i-1][k-1], Not(x[i]))
                )
            for i in range(1,n-1)
            ])
        )
    )

def exactly_one_he(bool_vars, name = ""):
    return And(at_most_one_he(bool_vars), at_least_one_he(bool_vars))

def Iff(A, B):
    return And(Implies(A, B), Implies(B, A))

### Solving

- Teams: $N$
- Periods: $P = N/2$
- Weeks: $W = N - 1$
- Round robin tournament: $rb$, a $P \times W$ matrix of tuples $t = (t_1, t_2)$ where $t_1 < t_2$
- Variables:
    - $index_{p,w,t}$: teams $rb[t][w]$ play in period $p$ of weeek $w$
    - $teams_{t_1, t_2, p}$: $t_1$ plays against $t_2$ in period $p$
- Constraints:
    - Every team plays at most once a week:
        - $\forall p \in P, w \in W.(\forall t \in P. exactly\_one(index_{p,w,t}))$
        - $\forall w \in W, t \in P.(\forall p \in P. exactly\_one(index_{p,w,t}))$
    - Every team plays between once and twice a week
        - $\forall p \in P, w \in W, t \in P.(index_{p,w,t} \leftrightarrow teams_{rb[t][w][0], rb[t][w][1], p})$
        - $\forall p \in P, t_1 \in N.(\forall t_2 \in N. (t_1 < t_2 \rightarrow at\_most\_2(teams_{t_1, t_2, p}))$

In [23]:
def solve_instance(rb, P, W, N):
    index = [[[Bool(f"index{p}_{w}_{t}") for t in range(P)] for w in range(W)] for p in range(P)]
    teams = [[[Bool(f"team{ti}_{tj}_{p}") for p in range(P)] for tj in range(N)] for ti in range(N)]

    s = Solver()
    
    # every team plays at most once every week
    for p in range(P):
       for w in range(W):
            s.add(exactly_one_he([index[p][w][t] for t in range(P)]))
    for w in range(W):
        for t in range(P):
            s.add(exactly_one_he([index[p][w][t] for p in range(P)]))

    # every team plays between once and twice in each period
    for p in range(P):
        for w in range(W):
            for t in range(P):
                s.add(Iff(index[p][w][t], teams[rb[t][w][0]][rb[t][w][1]][p]))
    for bp in range(P):
        s.add(at_most_k_seq([teams[0][t2][bp] for t2 in range(1, N)], 2))
        s.add(at_most_k_seq([teams[t2][-1][bp] for t2 in range(0, N-1)], 2))
        for t1 in range(1, N-1):
            s.add(at_most_k_seq([teams[t2][t1][bp] for t2 in range(t1)]+[teams[t1][t2][bp] for t2 in range(t1+1, N)], 2))

    s.check()
    model = s.model()


    res = []

    for p in range(P):
        matches = []
        for w in range(W):
            new_p = [model.evaluate(index[p][w][bp]) for bp in range(P)].index(True)
            matches.append(rb[new_p][w])

        res.append(matches)

    # for p in range(P):
    #     for w in range(W):
    #         print([model.evaluate(index[p][w][bp]) for bp in range(P)], end=", ")
    #     print()

    # for t1 in range(N):
    #     for t2 in range(N):
    #         if t1 < t2:
    #             print([model.evaluate(teams[t1][t2][bp]) for bp in range(P)], end=", ")
    #     print()


    return res

In [25]:
N = 6
P = N//2
W = N-1
S = 2

# round robin tournament with ordered slots
rb = []
for p in range(P):
    matches = []
    for w in range(W):
        if p == 0:
            matches.append(tuple(sorted([N-1, w])))
        else:
            matches.append(tuple(sorted([(p+w) % (N-1), (N-p+w-1)%(N-1)])))
    rb.append(matches)

res = solve_instance(rb, P, W, N)

for p in range(P):
    for w in range(W):
        print(f"({res[p][w][0]:<{int(math.log10(N)) + 1}},{res[p][w][1]:<{int(math.log10(N)) + 1}})", end=" ")
    print()

(0,5) (3,4) (0,4) (3,5) (1,2) 
(1,4) (1,5) (2,5) (2,4) (0,3) 
(2,3) (0,2) (1,3) (0,1) (4,5) 
